## 1️⃣ Setup - Clone Repository & Install Dependencies

In [ ]:
# Clone the repository (only once, to fixed location)
import os

REPO_PATH = '/content/Multimodal-Rare-Disease'

if not os.path.exists(REPO_PATH):
    !git clone https://github.com/ArshvirSk/Multimodal-Rare-Disease.git {REPO_PATH}
    print("✅ Repository cloned!")
else:
    print("✅ Repository already exists, pulling latest changes...")
    !cd {REPO_PATH} && git pull

# Change to repo directory
%cd {REPO_PATH}

# Verify
print(f"\n📁 Current directory: {os.getcwd()}")
print(f"📂 Contents: {os.listdir('.')[:10]}...")

In [ ]:
# Install dependencies
!pip install -q torch torchvision transformers pillow tqdm matplotlib seaborn
print("✅ Dependencies installed!")

In [ ]:
# Verify GPU is available
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ No GPU detected! Go to Runtime > Change runtime type > GPU")

## 2️⃣ Generate Augmented Training Data

Expands 50 base images → 550 augmented images

In [ ]:
# Verify source images exist first
import os
os.chdir('/content/Multimodal-Rare-Disease')

print("📷 Source images:")
!ls data/images/ | head -15
print(f"\n📊 Total source images: {len(os.listdir('data/images/'))}")

In [ ]:
# Generate augmented images
!python scripts/generate_augmented_data.py

In [ ]:
# Verify augmented data
from pathlib import Path

aug_dir = Path("data/images_augmented")
if aug_dir.exists():
    total_images = sum(1 for _ in aug_dir.rglob("*.png")) + sum(1 for _ in aug_dir.rglob("*.jpg"))
    syndromes = [d.name for d in aug_dir.iterdir() if d.is_dir()]

    print(f"✅ Total augmented images: {total_images}")
    print(f"✅ Syndromes: {len(syndromes)}")
    for s in sorted(syndromes):
        count = len(list((aug_dir / s).glob("*")))
        print(f"   • {s}: {count} images")
else:
    print("❌ Augmentation failed - directory not found!")

## 3️⃣ Train the Model

Training with:
- **Backbone:** ResNet50 (pretrained on ImageNet)
- **Transfer Learning:** Layers 1-3 frozen
- **Regularization:** Dropout 0.6, Label smoothing 0.12
- **Optimizer:** AdamW with Cosine Annealing LR

⏱️ Takes ~10-15 minutes on T4 GPU

In [ ]:
# Train the model
!python run_training.py --image-dirs data/images_augmented --epochs 60 --aug-factor 4 --batch-size 16

## 4️⃣ Make Predictions

Test the trained model on sample images

In [ ]:
# Test prediction on a sample image from augmented data
import os
from pathlib import Path

# Get a test image
aug_dir = Path("data/images_augmented")
test_img = list(aug_dir.rglob("*_orig.png"))[0]
print(f"Testing on: {test_img}\n")

!python predict.py --image "{test_img}"

In [ ]:
# Test on multiple syndromes
from pathlib import Path

aug_dir = Path("data/images_augmented")
syndromes = sorted([d for d in aug_dir.iterdir() if d.is_dir()])[:4]

for syn_dir in syndromes:
    orig_img = list(syn_dir.glob("*_orig.png"))[0]
    print(f"\n{'='*60}")
    print(f"Testing: {syn_dir.name}")
    !python predict.py --image "{orig_img}"

## 5️⃣ Upload Your Own Image

Upload a facial image to classify

In [ ]:
# Upload your own image
from google.colab import files
from IPython.display import Image, display

print("📤 Upload a facial image (JPG/PNG):")
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    print(f"\n✅ Uploaded: {filename}")
    
    # Display the image
    display(Image(filename, width=300))
    
    # Run prediction
    print("\n🔍 Running prediction...\n")
    !python predict.py --image "{filename}"

## 6️⃣ Visualize Sample Images

In [ ]:
# Visualize sample images from each syndrome
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path

fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('Sample Images from Each Syndrome', fontsize=14)

aug_dir = Path("data/images_augmented")
syndromes = sorted([d for d in aug_dir.iterdir() if d.is_dir()])

syndrome_names = {
    "SYN_CdLS": "Cornelia de Lange",
    "SYN_WBS": "Williams-Beuren",
    "SYN_NS": "Noonan",
    "SYN_KS": "Kabuki",
    "SYN_KBG": "KBG",
    "SYN_AS": "Angelman",
    "SYN_RSTS": "Rubinstein-Taybi",
    "SYN_SMS": "Smith-Magenis",
    "SYN_NBS": "Nicolaides-Baraitser",
    "SYN_22Q": "22q11.2 Deletion",
}

for idx, syn_dir in enumerate(syndromes[:10]):
    ax = axes[idx // 5, idx % 5]
    orig_imgs = list(syn_dir.glob("*_orig.png"))
    if orig_imgs:
        img = Image.open(orig_imgs[0])
        ax.imshow(img)
    name = syndrome_names.get(syn_dir.name, syn_dir.name)
    ax.set_title(name, fontsize=9)
    ax.axis('off')

plt.tight_layout()
plt.show()

## 📊 Model Architecture

```
┌─────────────────────────────────────────┐
│           Input Image (224x224)         │
└─────────────────┬───────────────────────┘
                  ▼
┌─────────────────────────────────────────┐
│     ResNet50 Backbone (ImageNet)        │
│     • Layers 1-3: Frozen                │
│     • Layer 4: Trainable                │
└─────────────────┬───────────────────────┘
                  ▼
┌─────────────────────────────────────────┐
│         Feature Vector (2048-d)         │
└─────────────────┬───────────────────────┘
                  ▼
┌─────────────────────────────────────────┐
│    Classification Head + Dropout        │
└─────────────────┬───────────────────────┘
                  ▼
┌─────────────────────────────────────────┐
│      Softmax → 10 Syndrome Classes      │
└─────────────────────────────────────────┘
```

---

## 🔗 Links

- **GitHub:** https://github.com/ArshvirSk/Multimodal-Rare-Disease
- **PDIDB Dataset:** https://github.com/WGLab/PDIDB
- **Orphadata:** https://www.orphadata.com

---

*Made with ❤️ for rare disease research*